In [124]:
from ipynb_style import clean, presentation, pres2

In [127]:
clean()
#presentation()
#pres2()

# Data Workflows in Stata and Python
<img src="Stata_Logo.svg" width="300" height="150" style="float: left; display: inline; margin: 10px">
<img src="python-logo-generic.svg" width="300" height="150" style="float: left; display: inline; margin: 10px">

# Data Workflows in Stata and Python

**Dejan Pavlic**, Education Policy Research Initiative, University of Ottawa

**Stephen Childs** (presenter), Office of Institutional Analysis, University of Calgary

<img src="uc-vert-rgb.png" width="173" height="129" style="float: left; display: inline; margin: 10px">
<img src="uottawa_ver_wg9.png" width="152" height="129" style="float: left; display: inline; margin: 25px">
<img src="epri_logo.jpg" width="524" height="129" style="float: left; display: inline; margin: 0px">

# Introduction

## About this talk

### Objectives

* know what Python is and what advantages it has
* know how Python can work with Stata

### Outline

* Introduction
    * Overall
    * Motivation
    * About Python

* Building Blocks
    * Pandas
    * Running Stata from Python
    * Python language features

* Workflows
    * ETL/Data Cleaning
    * Stata code generation
    * Processing Stata output

## About Me

* Started using Stata in grad school (2006).
* Using Python for about 3 years.
* Post-Secondary Education sector (in Canada)
    * University of Calgary - [Institutional Analysis](https://oia.ucalgary.ca/Contact)
    * [Education Policy Research Initiative](http://socialsciences.uottawa.ca/irpe-epri/eng/index.asp) - University of Ottawa (a Stata shop)

## Motivation

* Python is becoming very popular in the data world.
* Python skills are widely applicable.
* Python is powerful and flexible and will help you get more done, faster.

## About Python

### The Python Language

* General purpose programming language
* Name comes from Monty Python
* Python 2 vs. 3
* "batteries included"

### Scientific Python

<a href="http://pandas.pydata.org"><img src="pandas_logo.png"></a>

<a href="http://matplotlib.org/"><img src="matplotlib_logo.png">

<a href="http://www.numpy.org"><img src="numpy_logo.png"></a>


<center><h2><a href="http://scipy.org"><img src="scipyshiny_small.png" style="display: inline;"></a>SciPy</h2></center>

<a href="https://jupyter.org/"><img src="jupyter-sq-text.svg" height=200 width=200></a>

<a href="http://continuum.io/downloads"><img src="anaconda_logo_web.png"></a>

# Building Blocks

## Pandas

* General introduction
    * Origins - NumPy
    * Current popularity

* Key concepts
    * DataFrame
    * Series
    * index

In [54]:
from IPython.display import IFrame

In [55]:
import pandas as pd

In [56]:
auto = pd.read_stata('auto.dta')

In [57]:
import sys
import subprocess as sp
from threading import Thread
from queue import Queue, Empty

class Stpy:
    """A running instance of Stata as a subprocess.
    
    An instance of this class can be used to run Stata commands.
    The instance holds a persistence Stata process, so the memory
    is preserved between commands.

    """
    def __init__(self):
        """Constructor method for the stpy class.

        This starts Stata running on a separate thread, where it waits
        until commands are sent to it via the `write` method.

        """
        # Open stata as pipe; make a queue for non-blocking. Start the thread.
        self.proc = sp.Popen(['stata-mp'], stdin=sp.PIPE, stdout=sp.PIPE, bufsize=1)

        self.qu = Queue()

        self.thread = Thread(target = self.enqueue_output, args = (self.proc.stdout,
            self.qu))
        self.thread.daemon = True
        self.thread.start()

        # Read the initial stdout content.
        self.genout()

    def enqueue_output(self, out, queue):
        while True:
            # Read 1 byte at a time in thread.
            self.thread = out.read(1).decode()
            queue.put(self.thread)
        out.close()

    def buffer_output(self, sbuf=[]):
        # Try to read byte.
        try:  char = self.qu.get(timeout=1)
        except Empty:
            # Catch 2 x newline, followed by ". ". Stata's done with the output.
            if ''.join(sbuf) == "\n\n. ":
                return False # Done, Stata passes control back to user.
            # Pass if we are not done.
            else:
                pass # Waiting...
        else:
            # Keep a small internal rotating buffer which can catch the '\n\n. '
            # instruction.
            if len(sbuf) >= 4:
                del sbuf[0]
            sbuf.append(char)
            return(char)

    def genout(self):
        """Read the content of stdout."""
        ch = self.buffer_output()
        while ch:
            print(ch, end='')
            ch = self.buffer_output()

    def write(self, command):
        """Pass a stata command to the running Stata instance.
        
        Call this method with a string that you wish to pass to Stata.
        The method adds a newline to the end, then passes it to Stata.
        The output from Stata is returned via the standard output.
        """
        # Write command in bytes plus newline then flush.
        self.proc.stdin.write(bytes(command + "\n", 'ascii'))
        self.proc.stdin.flush()

        self.genout()

In [58]:
demo = Stpy()


  ___  ____  ____  ____  ____ (R)
 /__    /   ____/   /   ____/
___/   /   /___/   /   /___/   13.1   Copyright 1985-2013 StataCorp LP
  Statistics/Data Analysis            StataCorp
                                      4905 Lakeway Drive
     MP - Parallel Edition            College Station, Texas 77845 USA
                                      800-STATA-PC        http://www.stata.com
                                      979-696-4600        stata@stata.com
                                      979-696-4601 (fax)

2-user 2-core Stata network perpetual license:
       Serial number:  501306211345
         Licensed to:  Stephen Childs
                       Education Policy Research Initiative

Notes:
      1.  (-v# option or -set maxvar-) 5000 maximum variables
      2.  Command line editing disabled
      3.  Stata running in batch mode

running /Users/sechilds/Library/Application Support/Stata/profile.do ...

. 

In [59]:
demo.write('sysuse auto')

sysuse auto
(1978 Automobile Data)

. 

In [60]:
demo.write('describe')

describe

Contains data from /Applications/Stata/ado/base/a/auto.dta
  obs:            74                          1978 Automobile Data
 vars:            12                          13 Apr 2013 17:45
 size:         3,182                          (_dta has notes)
-------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
-------------------------------------------------------------------------------
make            str18   %-18s                 Make and Model
price           int     %8.0gc                Price
mpg             int     %8.0g                 Mileage (mpg)
rep78           int     %8.0g                 Repair Record 1978
headroom        float   %6.1f                 Headroom (in.)
trunk           int     %8.0g                 Trunk space (cu. ft.)
weight          int     %8.0gc                Weight (lbs.)
length          int     %8.0g                 Le

In [61]:
auto.shape

(74, 12)

In [62]:
auto.dtypes

make              object
price              int16
mpg                int16
rep78            float64
headroom         float32
trunk              int16
weight             int16
length             int16
turn               int16
displacement       int16
gear_ratio       float32
foreign         category
dtype: object

In [63]:
demo.write('summarize')

summarize

    Variable |       Obs        Mean    Std. Dev.       Min        Max
-------------+--------------------------------------------------------
        make |         0
       price |        74    6165.257    2949.496       3291      15906
         mpg |        74     21.2973    5.785503         12         41
       rep78 |        69    3.405797    .9899323          1          5
    headroom |        74    2.993243    .8459948        1.5          5
-------------+--------------------------------------------------------
       trunk |        74    13.75676    4.277404          5         23
      weight |        74    3019.459    777.1936       1760       4840
      length |        74    187.9324    22.26634        142        233
        turn |        74    39.64865    4.399354         31         51
displacement |        74    197.2973    91.83722         79        425
-------------+--------------------------------------------------------
  gear_ratio |        74    3.014865    .

In [64]:
auto.describe().T

,count,mean,std,min,25%,50%,75%,max
price,74,6165.256757,2949.495885,3291.00,4220.25,5006.500,6332.2500,15906.00
mpg,74,21.297297,5.785503,12.00,18.00,20.000,24.7500,41.00
rep78,69,3.405797,0.989932,1.00,3.00,3.000,4.0000,5.00
headroom,74,2.993243,0.845995,1.50,2.50,3.000,3.5000,5.00
trunk,74,13.756757,4.277404,5.00,10.25,14.000,16.7500,23.00
weight,74,3019.459459,777.193567,1760.00,2250.00,3190.000,3600.0000,4840.00
length,74,187.932432,22.266340,142.00,170.00,192.500,203.7500,233.00
turn,74,39.648649,4.399354,31.00,36.00,40.000,43.0000,51.00
displacement,74,197.297297,91.837219,79.00,119.00,196.000,245.2500,425.00
gear_ratio,74,3.014865,0.456287,2.19,2.73,2.955,3.3525,3.89


In [65]:
demo.write('list make price mpg rep78 foreign in 1/5')

list make price mpg rep78 foreign in 1/5

     +------------------------------------------------+
     | make            price   mpg   rep78    foreign |
     |------------------------------------------------|
  1. | AMC Concord     4,099    22       3   Domestic |
  2. | AMC Pacer       4,749    17       3   Domestic |
  3. | AMC Spirit      3,799    22       .   Domestic |
  4. | Buick Century   4,816    20       3   Domestic |
  5. | Buick Electra   7,827    15       4   Domestic |
     +------------------------------------------------+

. 

In [66]:
auto[['make', 'price', 'mpg', 'rep78', 'foreign']].head(5) 

,make,price,mpg,rep78,foreign
0,AMC Concord,4099,22,3,Domestic
1,AMC Pacer,4749,17,3,Domestic
2,AMC Spirit,3799,22,NaN,Domestic
3,Buick Century,4816,20,3,Domestic
4,Buick Electra,7827,15,4,Domestic


In [67]:
demo.write('\n'.join(['preserve',
                      'keep if foreign',
                      'list make price mpg rep78 in 1/5',
                      'restore']))

preserve

. keep if foreign
(52 observations deleted)

. list make price mpg rep78 in 1/5

     +----------------------------------+
     | make         price   mpg   rep78 |
     |----------------------------------|
  1. | Audi 5000    9,690    17       5 |
  2. | Audi Fox     6,295    23       3 |
  3. | BMW 320i     9,735    25       4 |
  4. | Datsun 200   6,229    23       4 |
  5. | Datsun 210   4,589    35       5 |
     +----------------------------------+

. restore

. 

In [68]:
auto[auto.foreign=='Foreign'][['make',
                               'price',
                               'mpg',
                               'rep78',
                               'foreign']].head(5)

,make,price,mpg,rep78,foreign
52,Audi 5000,9690,17,5,Foreign
53,Audi Fox,6295,23,3,Foreign
54,BMW 320i,9735,25,4,Foreign
55,Datsun 200,6229,23,4,Foreign
56,Datsun 210,4589,35,5,Foreign


In [69]:
demo.write('list make price mpg rep78 foreign if missing(rep78)')

list make price mpg rep78 foreign if missing(rep78)

     +-------------------------------------------------+
     | make             price   mpg   rep78    foreign |
     |-------------------------------------------------|
  3. | AMC Spirit       3,799    22       .   Domestic |
  7. | Buick Opel       4,453    26       .   Domestic |
 45. | Plym. Sapporo    6,486    26       .   Domestic |
 51. | Pont. Phoenix    4,424    19       .   Domestic |
 64. | Peugeot 604     12,990    14       .    Foreign |
     +-------------------------------------------------+

. 

In [70]:
auto[auto.rep78.isnull()]

,make,price,mpg,rep78,headroom,trunk,weight,length,turn,displacement,gear_ratio,foreign
2,AMC Spirit,3799,22,NaN,3.0,12,2640,168,35,121,3.08,Domestic
6,Buick Opel,4453,26,NaN,3.0,10,2230,170,34,304,2.87,Domestic
44,Plym. Sapporo,6486,26,NaN,1.5,8,2520,182,38,119,3.54,Domestic
50,Pont. Phoenix,4424,19,NaN,3.5,13,3420,203,43,231,3.08,Domestic
63,Peugeot 604,12990,14,NaN,3.5,14,3420,192,38,163,3.58,Foreign


In [71]:
demo.write('summarize price, detail')

summarize price, detail

                            Price
-------------------------------------------------------------
      Percentiles      Smallest
 1%         3291           3291
 5%         3748           3299
10%         3895           3667       Obs                  74
25%         4195           3748       Sum of Wgt.          74

50%       5006.5                      Mean           6165.257
                        Largest       Std. Dev.      2949.496
75%         6342          13466
90%        11385          13594       Variance        8699526
95%        13466          14500       Skewness       1.653434
99%        15906          15906       Kurtosis       4.819188

. 

In [72]:
auto.price.describe()

count       74.000000
mean      6165.256757
std       2949.495885
min       3291.000000
25%       4220.250000
50%       5006.500000
75%       6332.250000
max      15906.000000
Name: price, dtype: float64

In [73]:
auto.price.kurtosis(), auto.price.skew(), auto.price.var()

(2.0340476757316752, 1.6878409877224991, 8699525.9742687885)

In [74]:
auto.price.sort(inplace=False).head(4)

33    3291
13    3299
17    3667
67    3748
Name: price, dtype: int16

In [75]:
auto.price.sort(inplace=False).tail(4)

27    13466
26    13594
11    14500
12    15906
Name: price, dtype: int16

In [76]:
demo.write('tabulate foreign')

tabulate foreign

   Car type |      Freq.     Percent        Cum.
------------+-----------------------------------
   Domestic |         52       70.27       70.27
    Foreign |         22       29.73      100.00
------------+-----------------------------------
      Total |         74      100.00

. 

In [77]:
auto.foreign.value_counts()

Domestic    52
Foreign     22
dtype: int64

In [78]:
demo.write('tabulate rep78')

tabulate rep78

     Repair |
Record 1978 |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |          2        2.90        2.90
          2 |          8       11.59       14.49
          3 |         30       43.48       57.97
          4 |         18       26.09       84.06
          5 |         11       15.94      100.00
------------+-----------------------------------
      Total |         69      100.00

. 

In [79]:
auto.rep78.value_counts()

3    30
4    18
5    11
2     8
1     2
dtype: int64

In [80]:
demo.write('tabulate rep78 foreign')

tabulate rep78 foreign

    Repair |
    Record |       Car type
      1978 |  Domestic    Foreign |     Total
-----------+----------------------+----------
         1 |         2          0 |         2 
         2 |         8          0 |         8 
         3 |        27          3 |        30 
         4 |         9          9 |        18 
         5 |         2          9 |        11 
-----------+----------------------+----------
     Total |        48         21 |        69 


. 

In [81]:
pd.crosstab(auto.rep78, auto.foreign)

foreign,Domestic,Foreign
rep78,,
1,2,0
2,8,0
3,27,3
4,9,9
5,2,9


In [82]:
demo.write('tabulate rep78 foreign, missing')

tabulate rep78 foreign, missing

    Repair |
    Record |       Car type
      1978 |  Domestic    Foreign |     Total
-----------+----------------------+----------
         1 |         2          0 |         2 
         2 |         8          0 |         8 
         3 |        27          3 |        30 
         4 |         9          9 |        18 
         5 |         2          9 |        11 
         . |         4          1 |         5 
-----------+----------------------+----------
     Total |        52         22 |        74 


. 

In [83]:
pd.crosstab(auto.rep78.fillna(0), auto.foreign, margins=True)

foreign,Domestic,Foreign,All
rep78,,,
0.0,4,1,5
1.0,2,0,2
2.0,8,0,8
3.0,27,3,30
4.0,9,9,18
5.0,2,9,11
All,52,22,74


In [84]:
pd.crosstab(auto.rep78, auto.foreign).apply(lambda r: r/r.sum(), axis=0)

foreign,Domestic,Foreign
rep78,,
1,0.041667,0.000000
2,0.166667,0.000000
3,0.562500,0.142857
4,0.187500,0.428571
5,0.041667,0.428571


In [85]:
demo.write('summarize if foreign')

summarize if foreign

    Variable |       Obs        Mean    Std. Dev.       Min        Max
-------------+--------------------------------------------------------
        make |         0
       price |        22    6384.682    2621.915       3748      12990
         mpg |        22    24.77273    6.611187         14         41
       rep78 |        21    4.285714    .7171372          3          5
    headroom |        22    2.613636    .4862837        1.5        3.5
-------------+--------------------------------------------------------
       trunk |        22    11.40909    3.216906          5         16
      weight |        22    2315.909    433.0035       1760       3420
      length |        22    168.5455    13.68255        142        193
        turn |        22    35.40909    1.501082         32         38
displacement |        22    111.2273    24.88054         79        163
-------------+--------------------------------------------------------
  gear_ratio |        22    3.

In [86]:
auto[auto.foreign=='Foreign'].describe().T

,count,mean,std,min,25%,50%,75%,max
price,22,6384.681818,2621.915083,3748.00,4521.5000,5759.00,7067.5000,12990.00
mpg,22,24.772727,6.611187,14.00,21.0000,24.50,27.5000,41.00
rep78,21,4.285714,0.717137,3.00,4.0000,4.00,5.0000,5.00
headroom,22,2.613636,0.486284,1.50,2.5000,2.50,3.0000,3.50
trunk,22,11.409091,3.216906,5.00,9.2500,11.00,14.0000,16.00
weight,22,2315.909091,433.003454,1760.00,2025.0000,2180.00,2590.0000,3420.00
length,22,168.545455,13.682548,142.00,157.2500,170.00,174.7500,193.00
turn,22,35.409091,1.501082,32.00,34.2500,36.00,36.0000,38.00
displacement,22,111.227273,24.880537,79.00,92.5000,101.00,128.5000,163.00
gear_ratio,22,3.507273,0.296906,2.98,3.2325,3.61,3.7375,3.89


In [87]:
demo.write('by foreign: summarize')

by foreign: summarize

-------------------------------------------------------------------------------
-> foreign = Domestic

    Variable |       Obs        Mean    Std. Dev.       Min        Max
-------------+--------------------------------------------------------
        make |         0
       price |        52    6072.423    3097.104       3291      15906
         mpg |        52    19.82692    4.743297         12         34
       rep78 |        48    3.020833     .837666          1          5
    headroom |        52    3.153846    .9157578        1.5          5
-------------+--------------------------------------------------------
       trunk |        52       14.75    4.306288          7         23
      weight |        52    3317.115    695.3637       1800       4840
      length |        52    196.1346    20.04605        147        233
        turn |        52    41.44231    3.967582         31         51
displacement |        52    233.7115    85.26299         86        4

In [88]:
auto.groupby(by=auto.foreign).describe(percentiles=[]).T

foreign      Domestic                                                        \
                count         mean          std      min      50%       max   
displacement       52   233.711538    85.262993    86.00   231.00    425.00   
gear_ratio         52     2.806538     0.335960     2.19     2.75      3.58   
headroom           52     3.153846     0.915758     1.50     3.50      5.00   
length             52   196.134615    20.046054   147.00   200.00    233.00   
mpg                52    19.826923     4.743297    12.00    19.00     34.00   
price              52  6072.423077  3097.104279  3291.00  4782.50  15906.00   
rep78              48     3.020833     0.837666     1.00     3.00      5.00   
trunk              52    14.750000     4.306288     7.00    16.00     23.00   
turn               52    41.442308     3.967582    31.00    42.00     51.00   
weight             52  3317.115385   695.363740  1800.00  3360.00   4840.00   

foreign      Foreign                                                        
               count         mean          std      min      50%       max  
displacement      22   111.227273    24.880537    79.00   101.00    163.00  
gear_ratio        22     3.507273     0.296906     2.98     3.61      3.89  
headroom          22     2.613636     0.486284     1.50     2.50      3.50  
length            22   168.545455    13.682548   142.00   170.00    193.00  
mpg               22    24.772727     6.611187    14.00    24.50     41.00  
price             22  6384.681818  2621.915083  3748.00  5759.00  12990.00  
rep78             21     4.285714     0.717137     3.00     4.00      5.00  
trunk             22    11.409091     3.216906     5.00    11.00     16.00  
turn              22    35.409091     1.501082    32.00    36.00     38.00  
weight            22  2315.909091   433.003454  1760.00  2180.00   3420.00

In [89]:
demo.write('tabulate foreign, summarize(mpg)')

tabulate foreign, summarize(mpg)

            |      Summary of Mileage (mpg)
   Car type |        Mean   Std. Dev.       Freq.
------------+------------------------------------
   Domestic |   19.826923   4.7432972          52
    Foreign |   24.772727   6.6111869          22
------------+------------------------------------
      Total |   21.297297   5.7855032          74

. 

In [90]:
auto.groupby(by=auto.foreign)['mpg'].mean()

foreign
Domestic    19.826923
Foreign     24.772727
Name: mpg, dtype: float64

In [91]:
demo.write('ttest mpg, by(foreign)')

ttest mpg, by(foreign)

Two-sample t test with equal variances
------------------------------------------------------------------------------
   Group |     Obs        Mean    Std. Err.   Std. Dev.   [95% Conf. Interval]
---------+--------------------------------------------------------------------
Domestic |      52    19.82692     .657777    4.743297    18.50638    21.14747
 Foreign |      22    24.77273     1.40951    6.611187    21.84149    27.70396
---------+--------------------------------------------------------------------
combined |      74     21.2973    .6725511    5.785503     19.9569    22.63769
---------+--------------------------------------------------------------------
    diff |           -4.945804    1.362162               -7.661225   -2.230384
------------------------------------------------------------------------------
    diff = mean(Domestic) - mean(Foreign)                         t =  -3.6308
Ho: diff = 0                                     degrees of freedom 

In [92]:
from scipy.stats import ttest_ind
ttest_ind(auto.mpg[auto.foreign=='Foreign'],
          auto.mpg[auto.foreign=='Domestic'])

(3.6308484477318377, 0.0005254159338341992)

In [93]:
demo.write('correlate mpg weight')

correlate mpg weight
(obs=74)

             |      mpg   weight
-------------+------------------
         mpg |   1.0000
      weight |  -0.8072   1.0000


. 

In [94]:
auto.mpg.corr(auto.weight)

-0.80717485894244212

In [95]:
auto[['mpg','weight']].corr()

,mpg,weight
mpg,1.000000,-0.807175
weight,-0.807175,1.000000


In [96]:
demo.write('by foreign: correlate mpg weight')

by foreign: correlate mpg weight

-------------------------------------------------------------------------------
-> foreign = Domestic
(obs=52)

             |      mpg   weight
-------------+------------------
         mpg |   1.0000
      weight |  -0.8759   1.0000


-------------------------------------------------------------------------------
-> foreign = Foreign
(obs=22)

             |      mpg   weight
-------------+------------------
         mpg |   1.0000
      weight |  -0.6829   1.0000



. 

In [97]:
auto.groupby(by=auto.foreign)[['mpg','weight']].corr()

mpg    weight
foreign                            
Domestic mpg     1.000000 -0.875943
         weight -0.875943  1.000000
Foreign  mpg     1.000000 -0.682854
         weight -0.682854  1.000000

In [98]:
demo.write('regress mpg weight c.weight#c.weight i.foreign')

regress mpg weight c.weight#c.weight i.foreign

      Source |       SS       df       MS              Number of obs =      74
-------------+------------------------------           F(  3,    70) =   52.25
       Model |  1689.15372     3   563.05124           Prob > F      =  0.0000
    Residual |   754.30574    70  10.7757963           R-squared     =  0.6913
-------------+------------------------------           Adj R-squared =  0.6781
       Total |  2443.45946    73  33.4720474           Root MSE      =  3.2827

------------------------------------------------------------------------------
         mpg |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      weight |  -.0165729   .0039692    -4.18   0.000    -.0244892   -.0086567
             |
    c.weight#|
    c.weight |   1.59e-06   6.25e-07     2.55   0.013     3.45e-07    2.84e-06
             |
     foreign |
    Foreign  |    -2.2

In [99]:
import statsmodels.api as sm

In [100]:
auto['wtsq'] = auto.weight.apply(lambda x: x**2)

In [101]:
auto[['weight', 'wtsq']].head()

,weight,wtsq
0,2930,8584900
1,3350,11222500
2,2640,6969600
3,3250,10562500
4,4080,16646400


In [102]:
auto['is_foreign'] = pd.get_dummies(auto.foreign)['Foreign']

In [103]:
X = auto[['weight', 'wtsq', 'is_foreign']]
X = sm.add_constant(X)
res = sm.OLS(endog = auto.mpg, exog=X).fit()

In [104]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.691
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                     52.25
Date:                Sat, 25 Jul 2015   Prob (F-statistic):           7.68e-18
Time:                        21:43:49   Log-Likelihood:                -190.91
No. Observations:                  74   AIC:                             389.8
Df Residuals:                      70   BIC:                             399.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         56.5388      6.197      9.123      0.000        44.179    68.899
weight        -0.0166      0.004     -4.175      0.000        -0.024    -0.009
wtsq        1.591e-06   6.25e-07      2.546      0.013      3.45e-07  2.84e-06
is_foreign    -2.2035      1.059     -2.080      0.041        -4.316    -0.091
==============================================================================
Omnibus:                       37.797   Durbin-Watson:                   2.467
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               94.047
Skew:                           1.709   Prob(JB):                     3.78e-21
Kurtosis:                       7.337   Cond. No.                     1.76e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.76e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Stata Commands from Python

* Use the Stata command line
* Python's `subprocess` module runs each instance of Stata
* Each instance is a Python object
* Can send it commands with the `write()` method

In [105]:
stata = Stpy()


  ___  ____  ____  ____  ____ (R)
 /__    /   ____/   /   ____/
___/   /   /___/   /   /___/   13.1   Copyright 1985-2013 StataCorp LP
  Statistics/Data Analysis            StataCorp
                                      4905 Lakeway Drive
     MP - Parallel Edition            College Station, Texas 77845 USA
                                      800-STATA-PC        http://www.stata.com
                                      979-696-4600        stata@stata.com
                                      979-696-4601 (fax)

2-user 2-core Stata network perpetual license:
       Serial number:  501306211345
         Licensed to:  Stephen Childs
                       Education Policy Research Initiative

Notes:
      1.  (-v# option or -set maxvar-) 5000 maximum variables
      2.  Command line editing disabled
      3.  Stata running in batch mode

running /Users/sechilds/Library/Application Support/Stata/profile.do ...

. 

In [106]:
stata.write('sysuse auto')

sysuse auto
(1978 Automobile Data)

. 

In [107]:
stata.write('describe')

describe

Contains data from /Applications/Stata/ado/base/a/auto.dta
  obs:            74                          1978 Automobile Data
 vars:            12                          13 Apr 2013 17:45
 size:         3,182                          (_dta has notes)
-------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
-------------------------------------------------------------------------------
make            str18   %-18s                 Make and Model
price           int     %8.0gc                Price
mpg             int     %8.0g                 Mileage (mpg)
rep78           int     %8.0g                 Repair Record 1978
headroom        float   %6.1f                 Headroom (in.)
trunk           int     %8.0g                 Trunk space (cu. ft.)
weight          int     %8.0gc                Weight (lbs.)
length          int     %8.0g                 Le

Python strings have a `format()` method that allows you to substitute the contents of Python variables.

In [108]:
depvar = 'mpg'
indepvars = ['weight', 'wtsq', 'foreign']

In [109]:
'regress {depvar} {indepvars}'.format(depvar=depvar,
                                      indepvars=' '.join(indepvars))

'regress mpg weight wtsq foreign'

## Python Language Features

In [110]:
software = ['Stata', 'Python'] # list
person = {'name': 'Stephen Childs',
          'employer': 'University of Calgary',
          'city': 'Calgary',
          'province': 'Alberta'} #dictionary
software[0], person['name']

('Stata', 'Stephen Childs')

Python has some very useful and powerful language features. A good starting point is to look at some of the data structures built into the language. You can think of a Python list as a Stata macro list. In Python, lists can contain any type of object and can even contain different types in the same list. Lists are _ordered_.

Dictionaries are a very powerful data type. It lets you define a set of **keys** and related **values**. This is a very powerful and flexible data structure. The values are _unordered_.

In [111]:
def say_hello():
    print('Hello world!')

say_hello()    

Hello world!


In [112]:
def double(x):
    return x*2

double(2)

4

In [113]:
(lambda x: x**2)(8)

64

In [114]:
a = pd.Series(range(8))
a

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
dtype: int64

In [115]:
a.apply(lambda x: x**2)

0     0
1     1
2     4
3     9
4    16
5    25
6    36
7    49
dtype: int64

### Glossary

<table>
    <tr>
        <th>Stata</th>
        <th>Python</th>
    </tr>
    <tr>
        <td>macro</td>
        <td>variable</td>
    </tr>
    <tr>
        <td>macro list</td>
        <td>list</td>
    </tr>
    <tr>
        <td>variable</td>
        <td><code>pd.Series</code> or a column of a <code>pd.DataFrame</code></td>
    </tr>
        <tr>
        <td>data</td>
        <td><code>pd.DataFrame</code></td>
    </tr>
    <tr>
        <td><code>.dta</code> file</td>
        <td><ul>
            <li><a href="https://docs.python.org/3/library/pickle.html">Pickle</a> (though <a href="https://www.youtube.com/watch?v=7KnfGDajDQw">be careful!</a>)</li>
            <li><code>csv</code></li>
            <li><a href="https://www.hdfgroup.org/HDF5/">HDF5</a> (<a href="www.h5py.org">Python package</a>)</li>
            <li>etc&hellip;</li
        </ul></td>
    </tr>
        <tr>
        <td><code>ssc</code></td>
        <td><code>pip</code></td>
    </tr>
    <tr>
        <td><a href="https://ideas.repec.org/s/boc/bocode.html">Boston College Statistical Software Components (SSC) archive</a></td>
        <td><a href="https://pypi.python.org/pypi">PyPI - the Python Package Index</a> (<a href="https://www.youtube.com/watch?v=PPN3KTtrnZM">"The Cheeseshop"</a>)</td>
    </tr>
    <tr>
        <td><code>program</code></td>
        <td>function</td>
    </tr>
    <tr>
        <td><code>do</code> file</td>
        <td>module (<code>.py</code> file)</td>
    </tr>
    <tr>
        <td><code>ado</code> file</td>
        <td>package</td>
</table>

## Workflows

### ETL/Data Cleaning

* **advantages:**
    * wide variety of tools already available
    * create new data cleaning functions
* use pandas to prepare the data
    * organize code in separate files
    * move data into Stata when analysis file is ready

In [ ]:
"""Recode the PSIS Gender variable.

Input
-----
sd.raw.Gender.pd

Output
------
sd.Gender.pd

"""

import epandas as pd
import numpy as np

from eppdoc import eppdoc

# Load series.
d = eppdoc(__doc__)
s = d.read()

# Change to full labels.
s = s.replace({'F': 'Female', 'M': 'Male', np.nan: 'N/A'})

# Categorize and Order
s = s.astype('category')
s = s.cat.reorder_categories(['Female', 'Male', 'N/A'])

# Pickle.
d.writes(s)

e.g. You have a data source - a set of files - it is easy to write a Python sccript to turn that set of files into your analysis files.

### Stata Code Generation

* a less "fiddly" replacement for macros (see the `format` method above)
* deal with repitition and patterns

In [116]:
vars = ['mpg', 'rep78', 'headroom', 'trunk', 'weight',
        'length', 'turn', 'displacement',
        'gear_ratio', 'i.foreign']

In [117]:
from itertools import combinations

In [118]:
for x in combinations(vars, 2):
    print('regress price {vars}'.format(vars=' '.join(x)))

regress price mpg rep78
regress price mpg headroom
regress price mpg trunk
regress price mpg weight
regress price mpg length
regress price mpg turn
regress price mpg displacement
regress price mpg gear_ratio
regress price mpg i.foreign
regress price rep78 headroom
regress price rep78 trunk
regress price rep78 weight
regress price rep78 length
regress price rep78 turn
regress price rep78 displacement
regress price rep78 gear_ratio
regress price rep78 i.foreign
regress price headroom trunk
regress price headroom weight
regress price headroom length
regress price headroom turn
regress price headroom displacement
regress price headroom gear_ratio
regress price headroom i.foreign
regress price trunk weight
regress price trunk length
regress price trunk turn
regress price trunk displacement
regress price trunk gear_ratio
regress price trunk i.foreign
regress price weight length
regress price weight turn
regress price weight displacement
regress price weight gear_ratio
regress price weight i.

### Processing Stata Output

In [ ]:
import pandas as pd
import numpy as np
from epstata import Stpy
import predict_models

def predict_test(a, filename, model, key, iteration, results):
    a.write('use "{filename}", clear'.format(filename=filename))
    a.write('generate phat = .')
    tmp1 = NamedTemporaryFile(suffix='.dta')
    a.write('save {name}, replace'.format(name=tmp1.name))
    a.write('capture drop phat')
    a.write('set seed {iteration}'.format(iteration=iteration))
    a.write('generate cut = runiform()')
    a.write('logit Leaver {model} if cut < .5, iterate(200)'.format(
            model=model))
    a.write('predict phat if cut >= .5')
    a.write('keep if cut >= .5')
    a.write('keep ID Leaver phat')
    a.write('save {name}, replace'.format(name=tmp1.name))
    df = pd.read_stata(tmp1.name)
    df.set_index(df['ID'], inplace=True)
                df['Log Loss'] = -1 * (df['Leaver']*np.log(df['phat'])
            + (1-df['Leaver'])*np.log(1-df['phat']))
    results.write('{key} {iteration}: Log Loss: {logl}\n'.format(
            key=key,
            iteration=iteration,
            logl=df['Log Loss'].mean()))
    return df['Log Loss']

# Conclusion

* only an introduction to Python meant to whet your appitite
* show some possibilites
* Stata/Python integration is still a work in progress
* allow you to mix and match - replace part of your workflow with Python

In [119]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [120]:
IFrame(src='http://www.pyohio.org/', width=1000, height=400)

##Resources

* Brandon Rhodes [pandas tutorial](https://github.com/brandon-rhodes/pycon-pandas-tutorial) - PyCon Montréal 2015

## Contact

* The notebook will be up on github: [sechilds](https://github.com/sechilds)
* Twitter: [@sechilds](https://twitter.com/sechilds)
* E-mail: [Stephen.Childs@ucalgary.ca](mailto:Stephen.Childs@uCalgary.ca)